In [1]:
import os 
# import glob
# import json
import numpy as np
import pandas as pd
import SimpleITK as sitk
# import psycopg2
import matplotlib.pyplot as plt
# from numpy.linalg import inv
# import requests
# import shutil
# import re
# import sqlite3
import time
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


import sys
sys.path.append(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline')
from ccf_slice_registration_functions import *

sys.path.append(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline')
from soma_and_fiducial_pins import *

sys.path.append(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline')
from ccf_reg_database import *


Inputs

In [2]:
database_file = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline\ccf_reg.db'
out = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline\output' #folder to save slide transforms

#optional:
only_register_new = False #t/f should we only run cells that have not already been registered? (Default = True)
input_cells_to_register = False #t/f if there was a list of specimen ids to regiser (Default = False)
cell_specimen_id_file = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\ccf_slice_registration\pipeline\MSN_D1_Dataset_Sep_2023_ids.txt' #(Default = None) #NOTE don't need input_cells_to_register, can just see if this is none or a valid file. 
ccf_file =  r'\\allen\programs\celltypes\workgroups\mousecelltypes\_UPENN_fMOST\mouse_ccf_fixed_headers_um\average_template\average_template_10.nii.gz' #(Default = r'\\allen\programs\celltypes\workgroups\mousecelltypes\_UPENN_fMOST\mouse_ccf_fixed_headers_um\average_template\average_template_10.nii.gz')




Data wrangling 

In [3]:
#get list of all pins (somas and fiducials)
pins_path = get_soma_and_fiducial_pins()
pins = pd.read_csv(pins_path)

{'slide_specimen_id': 1069147202, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557045.04.09a'}
{'slide_specimen_id': 1069147202, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557045.04.09b'}
{'slide_specimen_id': 1069147202, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557045.04.09c'}
{'slide_specimen_id': 1102538034, 'specimen_name': 'Sst-IRES-Cre;Ai14-577302.07.04a', 'x': 4149.999999999999, 'y': 6388.183016393443, 'z': 8525.720459016393, 'structure_id': 0}
{'slide_specimen_id': 1068927614, 'specimen_name': 'Esr2-IRES2-Cre;Ai14-557044.05.09.03.01'}
{'slide_specimen_id': 1208614358, 'specimen_name': 'Curve'}
{'slide_specimen_id': 1262448162, 'specimen_name': 'Sst-IRES-Cre;Ai14-670419.09.03.02'}
{'slide_specimen_id': 1266114320, 'specimen_name': 'Gad2-IRES-Cre;Ai14-670842.06.02.01'}
{'slide_specimen_id': 1078776834, 'specimen_name': ' Htr3a-Cre_NO152;Ai14-563514.10.09.03'}
{'slide_specimen_id': 1186101158, 'specimen_name': 'C57BL6J-631344.05.03a'}
{'slide_specimen_id': 1186101158, 'specimen_name': 'C57BL6J-6

In [4]:
#pins contains fiducail pins (specimen name ends in a letter) and soma pins (specimen name ends in a number)
#break up pins into fiducial vs soma pin dataframes
fiducials_dict = {}
somas_dict = {}
for i, p in pins.iterrows():
    last_pin_char = p['specimen_name'][-1]
    if last_pin_char.isalpha(): fiducials_dict[i] = p #the last char of the pin name is a letter, so this is a fiducial (not a soma) pin 
    else: somas_dict[i] = p #the last char of the pin name is a number, so this is a soma (not a fiducial) pin

somas = pd.DataFrame.from_dict(somas_dict, orient='index').reset_index().drop(['index'], axis=1)
fiducials = pd.DataFrame.from_dict(fiducials_dict, orient='index').reset_index().drop(['index'], axis=1)


In [5]:
ensure_tables_exist(database_file) #make sure database is created every time we run this code. 

if only_register_new:
    #query db to get set of slices and cells that have already been registered
    # select slices that have not already been ccf registered 
    existing_slice_registrations = get_registered_slices(database_file)
    existing_slice_reg_ids = set(existing_slice_registrations['slice_id']) #slice ids - slices that have been registered
    slices_with_fiducials_ids = set(fiducials.slide_specimen_id) #slice ids - slices that could be registered
    slices_to_register_ids = list(slices_with_fiducials_ids - existing_slice_reg_ids) #slice ids - new slices to register
    fiducials = fiducials[fiducials['slide_specimen_id'].isin(slices_to_register_ids)] #fiducials for slices to register

In [6]:
#Load .txt file of spec ids to ccf reg 
if input_cells_to_register:
    cell_id_input = open(cell_specimen_id_file, "r")
    cell_ids_txt = cell_id_input.read()
    input_cell_ids = cell_ids_txt.split("\n")
    cell_id_input.close()
    while("" in input_cell_ids):input_cell_ids.remove("") #remove empty entries 
    input_cell_ids = list(set(input_cell_ids)) #remove duplicates
    input_cell_ids = [x.strip() for x in input_cell_ids] #remove leading/trailing whitespace 

    #get slice id for each cell 
    input_slice_ids = []
    for c in input_cell_ids:
        try: 
            c_name = get_name_by_id(c)
            s_name = c_name.rsplit('.',1)[0]
            s_id = get_id_by_name(s_name)
        except: continue
        input_slice_ids.append(s_id)
    input_slice_ids = list(set(input_slice_ids)) 

    #keep only these slices to register
    fiducials = fiducials[fiducials['slide_specimen_id'].isin(input_slice_ids)] #fiducials for slices to register

In [7]:
slices = fiducials.slide_specimen_id.unique() #slices to register

Load CCF

In [9]:
# read in CCF (with fixed headers)
ccf = sitk.ReadImage( ccf_file )

Transform slices to CCF

In [10]:
# #loop through all slices, registering them to the ccf 
# slices_with_issues = {}
# for specimen_id in slices:
#     specimen_id = int(specimen_id)
#     # specimen_id = 1108154695 #a good ex

#     #get slice name 
#     specimen_name = get_name_by_id(specimen_id)
#     print(specimen_name)

#     # try:

#     #make folder to store transform for this slice 
#     working_directory = os.path.join(out, specimen_name)
#     if not os.path.exists(working_directory):
#         os.mkdir(working_directory)

#     # read in pinning tool output - virtual slice definition and cell ccf locations
#     pinning_info = get_pinning(specimen_name)[4]

#     #TODO ensure that soma pins are inside the fiducial pins, otherwise record error and don't transform

#     # virtual_slice_to_ccf_transform: transform a 3D point (in micron) in the virtual slice to ccf (in micron)
#     # ccf_to_virtual_slice_transform: transform a 3D point in ccf (in micron) to virtual slice (in micron)
#     # (note: each point can have its own orientation - we are taking the first one only #TODO is this an issue? (swb))
#     virtual_slice_to_ccf_transform = initialize_transform( pinning_info['markups'][0]['orientation'] )
#     ccf_to_virtual_slice_transform = virtual_slice_to_ccf_transform.GetInverse()

#     # write out the virtual_slice_to_ccf_transform
#     file = os.path.join(working_directory, 'virtual_slice_to_ccf_transform.txt')
#     sitk.WriteTransform( virtual_slice_to_ccf_transform, file )


#     # generate a virtual slice from the transforms
#     #   virtual_slice_3d is a 3D volume with a single z slice
#     #   virtual_slice    is a 2D volume created by extracting out the single slice
#     virtual_slice_size = [1250,1250,1] # 3D volume with one slice
#     virtual_slice_spacing = ccf.GetSpacing()
#     virtual_slice_3d = sitk.Resample(ccf, 
#                                     virtual_slice_size, 
#                                     virtual_slice_to_ccf_transform, 
#                                     sitk.sitkLinear,
#                                     [0,0,0], 
#                                     virtual_slice_spacing, 
#                                     [1,0,0,0,1,0,0,0,1], 
#                                     0.0, 
#                                     ccf.GetPixelID())#, False )


#     virtual_slice = virtual_slice_3d[:,:,0] # 2D image version

#     # read in the 2D overview image and associate metadata
#     img_info = [list(get_20x_info(specimen_name))]

#     odf = pd.DataFrame(img_info, columns = ['specimen_name', 'sub_image_id', 'width', 'height', 'resolution', 'treatment_id'])
#     overview_info = odf.loc[0]
#     sub_image = odf['sub_image_id'].values[0]
#     get_20x_img(sub_image, specimen_name, working_directory)   
#     file = os.path.join(working_directory, '{}_overview.jpg'.format(specimen_name))
#     overview = sitk.ReadImage( file )
#     overview_spacing = [overview_info['resolution'],overview_info['resolution']]
#     overview.SetSpacing(overview_spacing)

#     # downsample the 2D overview image
#     downsampled_overview = sitk.BinShrink( sitk.VectorIndexSelectionCast(overview,0), [25,25])

#     # write the downsampled overview to file
#     file = os.path.join(working_directory,'downsampled_overview.nii.gz')
#     sitk.WriteImage( downsampled_overview, file, True )

#     # Read in drawn soma polygons to create matching landmarks set
#     df = get_soma_polygons(specimen_id)

#     # For each cell
#     #  - compute cell soma from polyline in pixels
#     #  - convert cell soma location to microns
#     #  - join with cell soma location in CCF
#     #  - compute cell soma location in virtual slice
#     df['center_pixel'] = [compute_center_from_polyline(p) for p in df['poly_coords']]
#     df['center_micron'] = [np.multiply(p,overview_spacing) for p in df['center_pixel']]

#     jdict = {}
#     for m in pinning_info['markups'] :
#         jdict[m['name']] = m['markup']['controlPoints'][0]['position']
        
#     df['ccf_coordinate'] = [jdict[p] for p in df['specimen_name']]
#     df['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in df['ccf_coordinate'] ]
#     #save df
#     df.to_csv(os.path.join(working_directory, 'alignment_output.csv'), index=False)


#     # for c in jdict.keys():
#     #     print('|{}|'.format(c))

#     lndmrks = get_landmark_ids(specimen_id)
#     ldf = pd.DataFrame()
#     for c in lndmrks:
#         # print(c)
#         this_lndmrk = get_landmark_location(c[0])
#         ldf = pd.concat([ldf, this_lndmrk])
        
#     ldf['center_pixel'] = [compute_center_from_polyline(p) for p in ldf['poly_coords']]
#     ldf['overview_coordinate'] = [np.multiply(p,overview_spacing) for p in ldf['center_pixel']]

#     ldf['ccf_coordinate'] = [jdict[p] for p in ldf['specimen_name']]
#     ldf['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in ldf['ccf_coordinate'] ]

#     if len(ldf) < 3:
#         print('\tnot enough fiducials')
#         # slices_with_issues[specimen_name] = 'not enough fiducials'
#         continue

#     # determine if slice was flipped, if so fix virtual to ccf transform 
#     if slice_flipped(ldf):
#         transform = virtual_slice_to_ccf_transform.GetParameters()
#         transform = list(transform)
#         idx = -1
#         if pinning_info['referenceView'].lower() == 'sagittal': idx = 2 #flip coronal axis in PIL to LPS transform 
#         if pinning_info['referenceView'].lower() == 'coronal': idx = 5 #flip saggital axis in RIA to LPS transform 
#         if idx > -1: transform[idx] = transform[idx]*-1 #flip axis
#         #TODO do I need to update one of the last three numbers in transform? 
#         transform = tuple(transform)
#         virtual_slice_to_ccf_transform.SetParameters(transform)

#     # write out the virtual_slice_to_ccf_transform
#     virtual_to_ccf_file = os.path.join(working_directory, 'virtual_slice_to_ccf_transform.txt')
#     sitk.WriteTransform( virtual_slice_to_ccf_transform, virtual_to_ccf_file )

#     # Concatentate landmarks (cell soma + additional) for registration
#     # virtual slice (fixed) landmarks
#     fixed_landmarks = [tuple(p) for p in df['virtual_slice_coordinate']]
#     fixed_landmarks.extend([tuple(p) for p in ldf['virtual_slice_coordinate']] )

#     # overview (moving) landmarks
#     moving_landmarks = [tuple(p) for p in df['center_micron']]
#     moving_landmarks.extend([tuple(p) for p in ldf['overview_coordinate']] )

#     # virtual_slice_to_overview_transform: transform a 2D point (in microns) in the virtual slice to overview image (in microns)
#     # overview_to_virtual_slice_transform: transform a 3D point in overview image (in microns) to virtual slice (in microns)
#     virtual_slice_to_overview_transform = \
#         sitk.LandmarkBasedTransformInitializer( sitk.AffineTransform(2), flatten(fixed_landmarks), flatten(moving_landmarks) )
#     overview_to_virtual_slice_transform = virtual_slice_to_overview_transform.GetInverse()

#     # write out the overview_to_virtual_slice_transform
#     overview_to_virtual_file = os.path.join(working_directory,'overview_to_virtual_slice_transform.txt')
#     sitk.WriteTransform( overview_to_virtual_slice_transform, overview_to_virtual_file )

#     # generate resampled overview image
#     resampled_overview = \
#         sitk.Resample(downsampled_overview, virtual_slice, virtual_slice_to_overview_transform, \
#                     sitk.sitkLinear, 0, downsampled_overview.GetPixelID())

#     # write the downsampled overview to file
#     file = os.path.join(working_directory,'resampled_overview.nii.gz')
#     sitk.WriteImage( resampled_overview, file, True ) 

#     #save transformation overview image
#     fig, axes = plt.subplots(nrows=1,ncols=3,figsize=(15,5)) 
#     visualize_landmarks(downsampled_overview, axes[0], moving_landmarks)
#     axes[0].set_title('20x with fiducials')
#     visualize_landmarks( virtual_slice, axes[1], fixed_landmarks)
#     axes[1].set_title('virtual slice with fiducials')
#     visualize_landmarks( resampled_overview, axes[2], fixed_landmarks)
#     axes[2].set_title('resampled 20x')
#     plt.savefig(os.path.join(working_directory, 'transformation_overview.jpg'))
#     # plt.show()
#     plt.clf()

#     #add slic_registrae to database
#     registration_time = os.path.getctime(overview_to_virtual_file)
#     registration_time = time.ctime(registration_time)
#     slice_row = (specimen_id, specimen_name, 1, overview_to_virtual_file, virtual_to_ccf_file, registration_time)
#     add_slice_registration(database_file, slice_row)


#     # except: 
#     #     print('\tissue with this slice {}'.format(specimen_id))
#     #     slices_with_issues[specimen_name] = 'issue with this slice'
#     #     continue


In [11]:
#loop through all slices, registering them to the ccf 
slices_with_issues = {}
for specimen_id in slices[1:2]:
    specimen_id = int(specimen_id)
    # specimen_id = 1108154695 #a good ex

    #get slice name 
    specimen_name = get_name_by_id(specimen_id)
    print(specimen_name)

# try:

    #make folder to store transform files for this slice 
    working_directory = os.path.join(out, specimen_name)
    if not os.path.exists(working_directory): os.mkdir(working_directory)


    # read in pinning tool output - virtual slice definition and cell ccf locations
    pinning_info = get_pinning(specimen_name)[4]

    #TODO ensure that soma pins are inside the fiducial pins, otherwise record error and don't transform

    # virtual_slice_to_ccf_transform: transform a 3D point (in micron) in the virtual slice to ccf (in micron)
    # ccf_to_virtual_slice_transform: transform a 3D point in ccf (in micron) to virtual slice (in micron)
    # (note: each point can have its own orientation - we are taking the first one only #TODO is this an issue? (swb))
    virtual_slice_to_ccf_transform = initialize_transform( pinning_info['markups'][0]['orientation'] )
    ccf_to_virtual_slice_transform = virtual_slice_to_ccf_transform.GetInverse()

    # generate a virtual slice from the transforms
    #   virtual_slice_3d is a 3D volume with a single z slice
    #   virtual_slice    is a 2D volume created by extracting out the single slice
    virtual_slice_size = [1250,1250,1] # 3D volume with one slice
    virtual_slice_spacing = ccf.GetSpacing()
    virtual_slice_3d = sitk.Resample(ccf, 
                                    virtual_slice_size, 
                                    virtual_slice_to_ccf_transform, 
                                    sitk.sitkLinear,
                                    [0,0,0], 
                                    virtual_slice_spacing, 
                                    [1,0,0,0,1,0,0,0,1], 
                                    0.0, 
                                    ccf.GetPixelID())#, False )


    virtual_slice = virtual_slice_3d[:,:,0] # 2D image version

    # read in the 2D overview image and associate metadata
    img_info = [list(get_20x_info(specimen_name))]

    odf = pd.DataFrame(img_info, columns = ['specimen_name', 'sub_image_id', 'width', 'height', 'resolution', 'treatment_id'])
    overview_info = odf.loc[0]
    sub_image = odf['sub_image_id'].values[0]
    image_path = os.path.join(working_directory, '{}_overview.jpg'.format(specimen_name))
    get_20x_img(sub_image, image_path)   
    file = os.path.join(working_directory, '{}_overview.jpg'.format(specimen_name))
    overview = sitk.ReadImage( file )
    overview_spacing = [overview_info['resolution'],overview_info['resolution']]
    overview.SetSpacing(overview_spacing)

    # downsample the 2D overview image
    downsampled_overview = sitk.BinShrink( sitk.VectorIndexSelectionCast(overview,0), [25,25])

    # Read in drawn soma polygons to create matching landmarks set
    df = get_soma_polygons(specimen_id)

    # For each cell
    #  - compute cell soma from polyline in pixels
    #  - convert cell soma location to microns
    #  - join with cell soma location in CCF
    #  - compute cell soma location in virtual slice
    df['center_pixel'] = [compute_center_from_polyline(p) for p in df['poly_coords']]
    df['center_micron'] = [np.multiply(p,overview_spacing) for p in df['center_pixel']]

    jdict = {}
    for m in pinning_info['markups']: jdict[m['name']] = m['markup']['controlPoints'][0]['position']
        
    df['ccf_coordinate'] = [jdict[p] for p in df['specimen_name']]
    df['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in df['ccf_coordinate'] ]

    lndmrks = get_landmark_ids(specimen_id)
    ldf = pd.DataFrame()
    for c in lndmrks:
        this_lndmrk = get_landmark_location(c[0])
        ldf = pd.concat([ldf, this_lndmrk])
        
    ldf['center_pixel'] = [compute_center_from_polyline(p) for p in ldf['poly_coords']]
    ldf['overview_coordinate'] = [np.multiply(p,overview_spacing) for p in ldf['center_pixel']]

    ldf['ccf_coordinate'] = [jdict[p] for p in ldf['specimen_name']]
    ldf['virtual_slice_coordinate'] = [ ccf_to_virtual_slice_transform.TransformPoint(p)[:2] for p in ldf['ccf_coordinate'] ]

    # ================= Error checking ===================

    #make sure there are at least three fiducials for this slice 
    if len(ldf) < 3:
        print('\tnot enough fiducials')
        slices_with_issues[specimen_name] = 'not enough fiducials'
        continue

    #make sure there are somas pinned for this cell, otherwise don't register since we can't confirm the somas are inside the fiducials 
    if len(df) < 1:
        print('\tno soma pins for this slice')
        slices_with_issues[specimen_name] = 'no soma pins'
        continue

    # ===================================================


    # determine if slice was flipped, if so fix virtual to ccf transform 
    if slice_flipped(ldf):
        transform = virtual_slice_to_ccf_transform.GetParameters()
        transform = list(transform)
        idx = -1
        if pinning_info['referenceView'].lower() == 'sagittal': idx = 2 #flip coronal axis in PIL to LPS transform 
        if pinning_info['referenceView'].lower() == 'coronal': idx = 5 #flip saggital axis in RIA to LPS transform 
        if idx > -1: transform[idx] = transform[idx]*-1 #flip axis
        #TODO do I need to update one of the last three numbers in transform? 
        transform = tuple(transform)
        virtual_slice_to_ccf_transform.SetParameters(transform)

    # Concatentate landmarks (cell soma + additional) for registration
    # virtual slice (fixed) landmarks
    fixed_landmarks = [tuple(p) for p in df['virtual_slice_coordinate']]
    fixed_landmarks.extend([tuple(p) for p in ldf['virtual_slice_coordinate']] )

    fixed_landmarks_plot_style = ['ro' for p in df['virtual_slice_coordinate']]
    fixed_landmarks_plot_style.extend(['r+' for p in ldf['virtual_slice_coordinate']] )

    # overview (moving) landmarks
    moving_landmarks = [tuple(p) for p in df['center_micron']]
    moving_landmarks.extend([tuple(p) for p in ldf['overview_coordinate']] )

    moving_landmarks_plot_style = ['ro' for p in df['center_micron']]
    moving_landmarks_plot_style.extend(['r+' for p in ldf['overview_coordinate']] )

    # virtual_slice_to_overview_transform: transform a 2D point (in microns) in the virtual slice to overview image (in microns)
    # overview_to_virtual_slice_transform: transform a 3D point in overview image (in microns) to virtual slice (in microns)
    virtual_slice_to_overview_transform = \
        sitk.LandmarkBasedTransformInitializer( sitk.AffineTransform(2), flatten(fixed_landmarks), flatten(moving_landmarks) )
    overview_to_virtual_slice_transform = virtual_slice_to_overview_transform.GetInverse()

    #determine whether this transform should be reviewed based on real to virtual slice transform param values 
    params = list(overview_to_virtual_slice_transform.GetParameters())
    if abs(params[0]) > 1.5 or params[1] > 1 or abs(params[2]) > 1 or params[3] < 0 or abs(params[4]) > 30000 or params[5] < 0 or params[5] > 5000: recommend_review = True
    else: recommend_review = False

    # generate resampled overview image
    resampled_overview = \
        sitk.Resample(downsampled_overview, virtual_slice, virtual_slice_to_overview_transform, \
                    sitk.sitkLinear, 0, downsampled_overview.GetPixelID())
    

    # =================== SAVE EVERTHING HERE ==================

    # write out the virtual_slice_to_ccf_transform
    virtual_to_ccf_file = os.path.join(working_directory, 'virtual_slice_to_ccf_transform.txt')
    sitk.WriteTransform(virtual_slice_to_ccf_transform, virtual_to_ccf_file )

    # write out the overview_to_virtual_slice_transform
    overview_to_virtual_file = os.path.join(working_directory,'overview_to_virtual_slice_transform.txt')
    sitk.WriteTransform(overview_to_virtual_slice_transform, overview_to_virtual_file )

    #save df (soma pin info)
    df.to_csv(os.path.join(working_directory, 'soma_alignment_output.csv'), index=False)

    #save ldf (fiducial pin info)
    ldf.to_csv(os.path.join(working_directory, 'fiducial_alignment_output.csv'), index=False)

    # write the downsampled overview to file
    file = os.path.join(working_directory,'downsampled_overview.nii.gz') 
    sitk.WriteImage(downsampled_overview, file, True )
    image_path = os.path.join(working_directory, '{}_downsampled_overview.jpg'.format(specimen_name))
    get_20x_img(sub_image, image_path, downsample=3)   

    # write the resampled overview to file
    file = os.path.join(working_directory,'resampled_overview.nii.gz')
    sitk.WriteImage(resampled_overview, file, True ) 

    #save transformation overview image
    fig, axes = plt.subplots(nrows=1,ncols=3,figsize=(15,5)) 
    visualize_landmarks(downsampled_overview, axes[0], moving_landmarks, moving_landmarks_plot_style)
    axes[0].set_title('20x with fiducials')
    visualize_landmarks( virtual_slice, axes[1], fixed_landmarks, fixed_landmarks_plot_style)
    axes[1].set_title('virtual slice with fiducials')
    visualize_landmarks( resampled_overview, axes[2], fixed_landmarks, fixed_landmarks_plot_style)
    axes[2].set_title('resampled 20x')
    plt.savefig(os.path.join(working_directory, 'transformation_overview.jpg'))
    plt.clf()

    # ==========================================================

    #record slice registration in database
    registration_time = os.path.getctime(overview_to_virtual_file)
    registration_time = time.ctime(registration_time)
    slice_row = (specimen_id, specimen_name, 1, overview_to_virtual_file, virtual_to_ccf_file, registration_time, recommend_review)
    add_slice_registration(database_file, slice_row)


# except: 
#     print('\tissue with this slice {}'.format(specimen_id))
#     slices_with_issues[specimen_name] = 'issue with this slice'
#     continue


Sst-IRES-Cre;Ai14-581875.06.04


<Figure size 1500x500 with 0 Axes>

In [12]:

slices_with_issues_df = pd.DataFrame.from_dict(slices_with_issues.items())
slices_with_issues_df.to_csv(addDateToFileName('slices_with_issues.csv'), index=False)


Transform cell morphologies to CCF

In [13]:
# == To transform morphology file into CCF space
# 1) download the morphology from LIMS
# 2) if needed convert coordinates to micron - using the resolution of the 63x image
# 3) if needed correct for shrinkage - this should only affect the z coordinate! -swb add this for autotrace only cells
# 4) translate the (x,y) coorinates of the morphology such that the soma node is in the corresponding position in the overview image
# 5) transform the morphology to match the virtual slice
# ---- apply "overview_to_virtual_slice_transform" to the (x,y) coordinates of the morphology
# 6) transform from virtual slice to CCF space
# ---- apply "virtual_slice_to_ccf_transform" to the (x,y,z) coordinates of the morphology
# 7) transform back to PIR (CCF coords) from LPS (ITK coords) -swb
# 8) shift cell so soma is in pinned location in ccf (from json) -swb

Inputs

In [14]:

if only_register_new:
    #select cells that have not already been ccf registered 
    existing_cell_registrations = get_registered_cells(database_file)
    existing_cell_reg_names = set(existing_cell_registrations['specimen_name']) #cell ids - cells that have all versions (manual, auto) registered
    cells_with_pinned_soma_names = set(somas.specimen_name.values) #cell names - cells that could have version(s) to register
    cells_to_register_names = list(cells_with_pinned_soma_names - existing_cell_reg_names) #cell n ames - new cells to register
    somas = somas[somas['specimen_name'].isin(cells_to_register_names)] #somas for cells to register

In [15]:
# select cells from slices that have been registered
existing_slice_registrations = get_registered_slices(database_file)
existing_slice_reg_ids = set(existing_slice_registrations['slice_id']) #slice ids - slices that have been registered
#only keep cells from slices that have been registered
somas = somas[somas['slide_specimen_id'].isin(existing_slice_reg_ids)]

In [16]:
if input_cells_to_register:
    #only register cells from the list of input spec ids
    input_cell_names = [get_name_by_id(x) for x in input_cell_ids]
    somas = somas[somas['specimen_name'].isin(input_cell_names)]

In [17]:
registered_cells = []
cells_with_issues = {}

for idx, cell in somas.iterrows():
    sp_name = cell['specimen_name']
    sl_name = sp_name.rsplit('.', 1)[0]

    print(sp_name)

    #reset variables for database entry 
    has_manual = 0
    manual_registered = 0
    manual_swc_path = "None"
    manual_registration_timestamp = "None"
    has_raw_autotrace = 0
    raw_autotrace_registered = 0
    raw_autotrace_swc_path = "None"
    raw_autotrace_registration_timestamp = "None"
    has_post_processed_step_14_autotrace = 0
    post_processed_step_14_autotrace_registered = 0
    post_processed_step_14_autotrace_swc_path = "None"
    post_processed_step_14_autotrace_registration_timestamp = "None"
    has_post_processed_step_22_autotrace = 0
    post_processed_step_22_autotrace_registered = 0
    post_processed_step_22_autotrace_swc_path = "None"
    post_processed_step_22_autotrace_registration_timestamp = "None"

    try:
        sp_id = get_id_by_name(sp_name) #cell id 
        sl_id = get_id_by_name(sl_name) #slice id

        #check if there's a transform for this slice 
        slice_path = os.path.join(out, sl_name)
        if not os.path.isfile(os.path.join(slice_path, 'overview_to_virtual_slice_transform.txt')): 
            cells_with_issues[sp_name] = 'Missing tranform file(s)'
            continue #go on to next cell 
        if not os.path.isfile(os.path.join(slice_path, 'virtual_slice_to_ccf_transform.txt')): 
            cells_with_issues[sp_name] = 'Missing tranform file(s)'
            continue #go on to next cell 
        if not os.path.isfile(os.path.join(slice_path, 'alignment_output.csv')): 
            cells_with_issues[sp_name] = 'Missing tranform file(s)'
            continue #go on to next cell 


        #get soma loc in 20x 
        alignment_output = pd.read_csv(os.path.join(slice_path, 'alignment_output.csv'))
        this_cell_alignment = alignment_output[alignment_output.specimen_name == sp_name]
        if len(this_cell_alignment) != 1: 
            cells_with_issues[sp_name] = 'No soma pin'
            print('\t no soma pin')
            continue
        lims_soma = this_cell_alignment['center_micron'].values
        lims_soma = lims_soma[0][1:-2].split(' ')
        lims_soma = [float(i) for i in lims_soma if len(i) > 0]

        #make folder to save registered swcs
        swc_path = os.path.join(slice_path, 'SWC')
        if not os.path.exists(swc_path): os.mkdir(swc_path)

        #load affine transforms
        overview_to_virtual_slice_transform = sitk.ReadTransform(os.path.join(slice_path, 'overview_to_virtual_slice_transform.txt'))
        virtual_slice_to_ccf_transform = sitk.ReadTransform(os.path.join(slice_path, 'virtual_slice_to_ccf_transform.txt'))

        #register lims gold standard swc 
        print('\tManual SWC:')
        swc_name = '{}'.format(sp_id)
        lims_path = list(get_swc_from_lims(str(sp_id)))[1]
        if not lims_path is None: 
            has_manual = 1 #there's a manual swc to register 
            try: 
                lims_path = edit_path(lims_path)
                morph = to_dict(lims_path)
                manual_swc_path = register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                            overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
                registered_cells = registered_cells + [sp_name+'_manual']
                print('\t\tMANUAL SWC REGISTERED!')
                manual_registered = 1 #manual swc was registered successfully 
                manual_registration_timestamp = os.path.getctime(manual_swc_path)
                manual_registration_timestamp = time.ctime(manual_registration_timestamp)

            except: 
                print('\t\terror registering manual swc')

        #register autotrace raw swc
        print('\tRaw autotrace SWC:')
        swc_name = '{}_autotrace_raw'.format(sp_id)
        raw_path = get_autotrace_raw_path(sp_id)
        if os.path.isfile(raw_path):
            has_raw_autotrace = 1
            try: 
                morph = to_dict(raw_path)
                morph = convert_pixel_to_um_dictnrn(morph, sp_id)
                raw_autotrace_swc_path = register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
                registered_cells = registered_cells + [sp_name+'_raw']
                print('\t\tRAW AUTOTRACE SWC REGISTERED!')
                raw_autotrace_registered = 1
                raw_autotrace_registration_timestamp = os.path.getctime(raw_autotrace_swc_path)
                raw_autotrace_registration_timestamp = time.ctime(raw_autotrace_registration_timestamp)
            except: print('\t\terror registering raw autotrace swc')


        #register autotrace pp14 swc
        print('\tPost processed step 14 autotrace SWC:')
        swc_name = '{}_autotrace_pp14'.format(sp_id)
        pp14_path = get_autotrace_pp_path(sp_id, 14)
        if os.path.isfile(pp14_path):
            has_post_processed_step_14_autotrace = 1
            try: 
                morph = to_dict(pp14_path)
                morph = convert_pixel_to_um_dictnrn(morph, sp_id)
                post_processed_step_14_autotrace_swc_path = register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
                registered_cells = registered_cells + [sp_name+'_pp14']
                print('\t\tPP14 AUTOTRACE SWC REGISTERED!')
                post_processed_step_14_autotrace_registered = 1
                post_processed_step_14_autotrace_registration_timestamp = os.path.getctime(post_processed_step_14_autotrace_swc_path) 
                post_processed_step_14_autotrace_registration_timestamp = time.ctime(post_processed_step_14_autotrace_registration_timestamp)
            except: print('\t\terror registering pp14 autotrace swc')

        
        #register autotrace pp22 swc
        print('\tPost processed step 22 autotrace SWC:')
        swc_name = '{}_autotrace_pp22'.format(sp_id)
        pp22_path = get_autotrace_pp_path(sp_id, 22)
        if os.path.isfile(pp22_path):
            has_post_processed_step_22_autotrace = 1
            try:
                morph = to_dict(pp22_path)
                morph = convert_pixel_to_um_dictnrn(morph, sp_id)
                post_processed_step_22_autotrace_swc_path = register_morph(sp_name, sp_id, lims_soma, morph, swc_path, swc_name, somas, 
                        overview_to_virtual_slice_transform, virtual_slice_to_ccf_transform)
                registered_cells = registered_cells + [sp_name+'_pp22']
                print('\t\tPP22 AUTOTRACE SWC REGISTERED!')
                post_processed_step_22_autotrace_registered = 1
                post_processed_step_22_autotrace_registration_timestamp = os.path.getctime(post_processed_step_22_autotrace_swc_path)
                post_processed_step_22_autotrace_registration_timestamp = time.ctime(post_processed_step_22_autotrace_registration_timestamp)
            except: print('\t\terror registering pp22 autotrace swc')

        #add cell to db 
        cell_row = (sp_id, sp_name, sl_id, 
                    has_manual, manual_registered, manual_swc_path, manual_registration_timestamp, 
                    has_raw_autotrace, raw_autotrace_registered, raw_autotrace_swc_path, raw_autotrace_registration_timestamp, 
                    has_post_processed_step_14_autotrace, post_processed_step_14_autotrace_registered, post_processed_step_14_autotrace_swc_path, post_processed_step_14_autotrace_registration_timestamp, 
                    has_post_processed_step_22_autotrace, post_processed_step_22_autotrace_registered, post_processed_step_22_autotrace_swc_path, post_processed_step_22_autotrace_registration_timestamp)
        add_cell_registration(database_file, cell_row)
        
    except: continue

C57BL6J-617127.16.09.01
Vip-IRES-Cre;Ai14-579327.07.04.03
Rorb-IRES2-Cre-neo;Ai14-571899.05.10.02
Sst-IRES-Cre;Ai14-572054.06.10.01
Gad2-IRES-Cre;Ai14-600206.05.03.02
C57BL6J-631343.08.04.02
C57BL6J-631343.08.04.04
Sst-IRES-Cre;Ai14-572056.04.10.03
Gad2-IRES-Cre;Ai14-574192.10.09.01
Vip-IRES-Cre;Ai14-579326.07.03.02
Vip-IRES-Cre;Ai14-580199.07.02.02
C57BL6J-581851.08.03.02
C57BL6J-581850.06.03.01
C57BL6J-581850.06.03.04
Gad2-IRES-Cre;Ai14-580680.07.04.03
Chat-IRES-Cre-neo;Ai14-585474.07.03.02
Rorb-IRES2-Cre-neo;Ai14-611981.05.03.02
Ctgf-T2A-dgCre;Ai14-553529.08.06.02
Vipr2-IRES2-Cre;Slc32a1-IRES2-FlpO;Ai65-562071.12.09.03
Vip-IRES-Cre;Ai14-579327.06.03.04
Vip-IRES-Cre;Ai14-609322.06.03.04
Vip-IRES-Cre;Ai14-609322.06.04.02
Vip-IRES-Cre;Ai14-609322.06.04.04
Vip-IRES-Cre;Ai14-609322.06.04.05
Rorb-IRES2-Cre-neo;Ai14-611981.07.03.01
Vip-IRES-Cre;Ai14-614426.05.03.01
C57BL6J-617129.15.09.02
Pvalb-IRES-Cre;Ai14-609302.05.04.02
Rbp4-Cre_KL100;Ai14-613769.06.04.03
Slc17a6-IRES-Cre;Ai14-622107.0

In [18]:
cells_with_issues_df = pd.DataFrame.from_dict(list(cells_with_issues.items())) 
cells_with_issues_df.to_csv(addDateToFileName('cells_with_issues.csv'), index=False)

In [19]:
registered_cells_df = pd.DataFrame(registered_cells)
registered_cells_df.to_csv(addDateToFileName('registered_cells.csv'), index=False)

In [4]:
get_name_by_id(1069241269)

'Esr2-IRES2-Cre;Ai14-557045.05.09.02'